<a href="https://colab.research.google.com/github/shred-code/vedas-rag-chatbot/blob/main/embedder_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

input_path = "/content/drive/MyDrive/vedas-rag/chunks/rigveda_book1.json"
with open(input_path, "r") as f:
    hymns = json.load(f)


In [ ]:
texts = []
metadatas = []
ids = []

for hymn in hymns:
    # embed
    content = f"{hymn['title']}\n" + " ".join(hymn["verses"])
    texts.append(content)

    # Metadata to store alongside
    metadatas.append({
        "id": hymn["id"],
        "veda": hymn["veda"],
        "book": hymn["book"],
        "deity": hymn["deity"],
        "title": hymn["title"]
    })

    # ID
    ids.append(hymn["id"])

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(texts)
embedding_array = np.array(embeddings).astype("float32")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install faiss-cpu
import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.4 MB/s eta 0:00:00


In [ ]:
dim = embedding_array.shape[1]
idx = faiss.IndexFlatL2(dim)

idx.add(embedding_array)
faiss.write_index(idx, "/content/drive/MyDrive/vedas-rag/embeddings/rigveda_book1.index")

In [ ]:
# testing retrieval
query = "Who is Agni?"
q_v = model.encode([query]).astype('float32')
D, I = idx.search(q_v, k=3)
for idx in I[0]:
    print(f"→ {metadatas[idx]['title']}")
    print(texts[idx][:300], "\n---")

→ HYMN LXX. Agni.
HYMN LXX. Agni.
1. MAY we, the pious, win much food by prayer, may Agni with fair light pervade each act,— He the observer of the heavenly laws of Gods, and of the race of mortal man. He who is germ of waters, germ of woods, germ of all things that move not and that move,— To him even in the rock an 
---
→ HYMN XII. Agni.
HYMN XII. Agni.
WE choose Agni the messenger, the herald, master of all wealth, Well skilled in this our sacrifice. With callings ever they invoke Agni, Agni, Lord of the House, Oblation-bearer, much beloved. Bring the Gods hither, Agni, born for him who strews the sacred grass: Thou art our herald, 
---
→ HYMN I. Agni.
HYMN I. Agni.
I Laud Agni, the chosen Priest, God, minister of sacrifice, The hotar, lavishest of wealth. Worthy is Agni to be praised by living as by ancient seers. He shall bring hitherward the Gods. Through Agni man obtaineth wealth, yea, plenty waxing day by day, Most rich in heroes, glorious. A 
---
